## DS 5110 Final Project - Hospital Mortality
### By: Elena Tsvetskova, Brian Chae, Ryan Viti (rrv7eb)

In [21]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

spark = SparkSession.builder \
        .master("local") \
        .appName("Team 3 Final Project") \
        .getOrCreate()

filename = "hospital_mortality.csv"

data = spark.read.csv(filename, inferSchema=True, header=True)
data.show(2)

+------------+----------+-----------+---+-----+----------------+---------+------+------+----------------+------+-------------+------------+----------------+------+------------------+-------------------+---------------------+----------+---------------+----------------+-----------------+-----------------+-----------------+----------------+----------+---------------+-----------+-----------------+-------------+-------------+-------------------------+-------------------------+----------------+----------------+----------+----------+----------------------+----------------------+---------------+---------------+-----------+-----------+------------+------------+------------------------+------------------------+-----------+-----------+-------------+-------------+-------------------------+-------------------------+----------------+----------------+----------+----------+----------------------+----------------------+---------------+---------------+-----------+-----------+------------+------------+--

In [2]:
data.printSchema()

root
 |-- encounter_id: integer (nullable = true)
 |-- patient_id: integer (nullable = true)
 |-- hospital_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- elective_surgery: integer (nullable = true)
 |-- ethnicity: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- height: double (nullable = true)
 |-- icu_admit_source: string (nullable = true)
 |-- icu_id: integer (nullable = true)
 |-- icu_stay_type: string (nullable = true)
 |-- icu_type: string (nullable = true)
 |-- pre_icu_los_days: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- apache_2_diagnosis: integer (nullable = true)
 |-- apache_3j_diagnosis: double (nullable = true)
 |-- apache_post_operative: integer (nullable = true)
 |-- arf_apache: integer (nullable = true)
 |-- gcs_eyes_apache: integer (nullable = true)
 |-- gcs_motor_apache: integer (nullable = true)
 |-- gcs_unable_apache: integer (nullable = true)
 |-- gcs_verbal_apache

## Data Cleaning and EDA

In [3]:
# empty columns and unique identifiers unnecessary for analysis

data = data.drop("_c83", "encounter_id", "patient_id", "hospital_id", "icu_id")

# Subsetting data by type of data for EDA purposes
integer_data = data.select([data.dtypes[i][0] for i in range(len(data.dtypes)) if (data.dtypes[i][1] == 'int') and (data.select(data[i]).distinct().count() >= 10)])

for i in integer_data.columns:
    integer_data.describe(i).show()
    integer_data.select(F.percentile_approx(i, [0.25, 0.75], 100000).alias("quantiles")).show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|             23308|
|   mean| 62.60936159258624|
| stddev|16.739045957199462|
|    min|                16|
|    max|                89|
+-------+------------------+

+---------+
|quantiles|
+---------+
| [53, 76]|
+---------+

+-------+------------------+
|summary|apache_2_diagnosis|
+-------+------------------+
|  count|             24025|
|   mean|187.11779396462018|
| stddev| 85.93823690825164|
|    min|               101|
|    max|               308|
+-------+------------------+

+----------+
| quantiles|
+----------+
|[113, 301]|
+----------+

+-------+-----------------+
|summary|heart_rate_apache|
+-------+-----------------+
|  count|            24274|
|   mean|98.78104144351981|
| stddev|31.05756946762657|
|    min|               30|
|    max|              178|
+-------+-----------------+

+---------+
|quantiles|
+---------+
|[82, 119]|
+---------+

+-------+-----------------+
|summar

In [23]:
hospital_deaths = data.select("hospital_death").filter(col("hospital_death")==1 | col("hospital_death")).groupBy("hospital_death").count()
# hospital_deaths = hospital_deaths.withColumn('percent', )
hospital_deaths.show()

Py4JError: An error occurred while calling o1132.or. Trace:
py4j.Py4JException: Method or([class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.base/java.lang.Thread.run(Thread.java:829)



In [20]:
data.hospital_deaths.show()

AttributeError: 'DataFrame' object has no attribute 'hospital_deaths'

In [8]:
data.select("hospital_death").show()

+--------------+
|hospital_death|
+--------------+
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             1|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
|             0|
+--------------+
only showing top 20 rows

